###  RNN
- 동작방식 : 순차적 적립 방식(=기억, 누적, 압축)
- 문제점 : 입력이 길어질수록 이전 입력에 대한 정보소실=기울기소실, 번역에 사용하기 어려움
     - 대안으로 문장 전체를 보고 나서 생성하는 구조 필요 ((Seq2Seq)
     - <a href="https://proceedings.neurips.cc/paper/2014/file/a14ac55a4f27472c5d894ec1c3c743d2-Paper.pdf" target="_blank"> Sequence to Sequence Learning 논문 </a> 
- 동작방식 : 순차적 적립 방식(=기억, 누적, 압축)
- 문제점 : 입력이 길어질수록 이전 입력에 대한 정보소실=기울기소실, 번역에 사용하기 어려움
     - 대안으로 문장 전체를 보고 나서 생성하는 구조 필요 ((Seq2Seq)
     - <a href="https://wikidocs.net/24996" target="_blank"> Sequence to Sequence 설명 </a> ,  <a href="https://reniew.github.io/35/" target="_blank"> Sequence to Sequence 추가설명 </a> 
     - 단순한 RNN은 긴 입력에 대한 정보를 학습시키기 어렵다.이에 <a href="https://dgkim5360.tistory.com/entry/understanding-long-short-term-memory-lstm-kr" target="_blank"> LSTM </a> 을 사용하고자하였다.
         - 일반 시퀀스 학습을 위한 간단한 전략은 하나의 RNN을 사용하여 입력 시퀀스를 고정 크기의 벡터에 매핑한 다음 다른 RNN을 사용하여 벡터를 목표 시퀀스에 매핑하는 것이다(이 접근법은 Cho 등 5에서도 채택되었다). RNN이 모든 관련 정보를 제공하기 때문에 원칙적으로 작동할 수 있지만, 결과적으로 장기 의존성 14, 4 (그림 1) 16, 15로 인해 RNN을 훈련시키기는 어려울 것이다. 그러나 LSTM(Long Short-Term Memory) 16은 시간 의존성이 긴 문제를 학습하는 것으로 알려져 있으므로 LSTM은 이 설정에서 성공할 수 있다.

## [Sequence to Sequence ] 문제
- 내부가 보이지 않는 커다란 블랙박스에서 점차적으로 확대해가는 방식
- 인코더와 디코더 아키텍처로 구성되어 있다.
    - 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 <MARK>컨텍스트 벡터(context vector)</MARK>라고 합니다. 
    - 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송
    - 디코더에서 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력한다.
 - Seq2Seq 모델은 LSTM의 한계와 마찬가지로 입력 문장이 매우 길면 효율적으로 학습하지 못합니다.

### Seq2Seq의 추론 방식

- 입력 시퀀스를 상태 벡터들로 바꿉니다.
- 크기가 1인 목표 시퀀스로 시작합니다.(시퀀스의 시작 문자에만 해당)
- 상태 벡터들과 크기가 1인 목표 시퀀스를 decoder에 넣어 다음 문자에 대한 예측치를 생성합니다.
- 이런 예측치들을 사용해 다음 문자의 표본을 뽑습니다.(간단하게 argmax를 사용)
- 목표 시퀀스에 샘플링된 문자를 붙입니다.
- 시퀀스 종료 문자를 생성하거나 끝 문자에 도달할 때까지 앞의 과정을 반복합니다.

<img src="https://blog.kakaocdn.net/dn/BKXJt/btqCd1I0OTv/DujVTXOZ54SwKomoSpjRCK/img.gif" srcset="https://blog.kakaocdn.net/dn/BKXJt/btqCd1I0OTv/DujVTXOZ54SwKomoSpjRCK/img.gif" width="100%" data-origin-width="1440" data-origin-height="616">

[입력과 출력의 "establish communication"에 대해 설명]
- 입력 abc와 출력 def에 대해 a->d, b->e, c->f의 관계를 가진다면 입력을 cba로 뒤집어 관련 있는 단어끼리 같은 거리를 갖게 한다. 이 단순한 변형은 SGD가 입출력 간의 구조적 관계를 파악하는 것을 용이하게 하고 LSTM에 대해 성능을 Boost하는 효과가 있다

어텐션 메커니즘(Attention)
-  문맥을 더 잘 반영하는 벡터를 생성하는 메커니즘

<img src="https://blog.kakaocdn.net/dn/bkL0yX/btqCdgGnHVl/9PYdhz6L6xspMpke7zjBHk/img.gif" srcset="https://blog.kakaocdn.net/dn/bkL0yX/btqCdgGnHVl/9PYdhz6L6xspMpke7zjBHk/img.gif" data-filename="seq2seq_7.gif" data-origin-width="1432" data-origin-height="628">

## Sequence to Sequence 구현

#### LSTM Encoder

In [1]:
import tensorflow as tf

class Encoder(tf.keras.Model): #압축한 컨텍스트 벡터
  def __init__(self, vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(enc_units) # return_sequences 매개변수를 기본값 False로 전달

  def call(self, x):
    print("입력 Shape:", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    output = self.lstm(x)
    print("LSTM Layer의 Output Shape:", output.shape)

    return output

print("슝~")

슝~


In [2]:
vocab_size = 30000
emb_size = 256
lstm_size = 512
batch_size = 1
sample_seq_len = 3

print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [3]:
encoder = Encoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

sample_output = encoder(sample_input)    # 컨텍스트 벡터(입력층에서 나온 은닉층의 마지막 벡터)로 사용할 인코더 LSTM의 최종 State값

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
LSTM Layer의 Output Shape: (1, 512)


### LSTM Decoder

In [4]:
# Encoder 구현에 사용된 변수들을 이어 사용함에 유의!

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.lstm = tf.keras.layers.LSTM(dec_units,
                                     return_sequences=True) # return_sequences 매개변수를 True로 설정
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.softmax = tf.keras.layers.Softmax(axis=-1)

  def call(self, x, context_v):  # 디코더의 입력 x와 인코더의 컨텍스트 벡터를 인자로 받는다. 
    print("입력 Shape:", x.shape)

    x = self.embedding(x)
    print("Embedding Layer를 거친 Shape:", x.shape)

    context_v = tf.repeat(tf.expand_dims(context_v, axis=1),
                          repeats=x.shape[1], axis=1)
    x = tf.concat([x, context_v], axis=-1)  # 컨텍스트 벡터를 concat 해준다
    print("Context Vector가 더해진 Shape:", x.shape)

    x = self.lstm(x)
    print("LSTM Layer의 Output Shape:", x.shape)

    output = self.fc(x)
    print("Decoder 최종 Output Shape:", output.shape)

    return self.softmax(output)

print("슝~")

슝~


In [5]:
print("Vocab Size: {0}".format(vocab_size))
print("Embedidng Size: {0}".format(emb_size))
print("LSTM Size: {0}".format(lstm_size))
print("Batch Size: {0}".format(batch_size))
print("Sample Sequence Length: {0}\n".format(sample_seq_len))

Vocab Size: 30000
Embedidng Size: 256
LSTM Size: 512
Batch Size: 1
Sample Sequence Length: 3



In [6]:
decoder = Decoder(vocab_size, emb_size, lstm_size)
sample_input = tf.zeros((batch_size, sample_seq_len))

dec_output = decoder(sample_input, sample_output)  # Decoder.call(x, context_v) 을 호출

입력 Shape: (1, 3)
Embedding Layer를 거친 Shape: (1, 3, 256)
Context Vector가 더해진 Shape: (1, 3, 768)
LSTM Layer의 Output Shape: (1, 3, 512)
Decoder 최종 Output Shape: (1, 3, 30000)


###  Attention! (1) Bahdanau Attention
- Attention 메커니즘의 이모저모를 살펴보기
- <a href="https://arxiv.org/pdf/1409.0473.pdf" target="_blank"> Bahdanau 의 Attention 논문 보기 </a> 
- Attention Map의 시각화를 통해 모델의 안정성을 점검할 수 있고, 모델이 의도와 다르게 작동될 경우 그 원인을 찾아내는 데에 이용될 수 있다.

[seq2seq vs attnseq2seq 차이] 

![seq2seq_attnseq2seq](https://user-images.githubusercontent.com/70866993/137127166-437955c6-c2b1-499f-9e44-92041ea6419b.png)

- attention이 있는 경우엔 바로 context vector c에 첨자 i가 붙어있다는 점만 다르다
    - 디코더에서 다시한번 인코더의 실제입력값을 참고한다.

[Bahdanau_Attention] = 이름 : concat
현재 타임 스텝의 디코더 Hidden State Vector $s_t$ 를 구하기 위해서 현재 타임 스텝의 Context Vector  $c_t$  및 이전 타임 스텝의 Hidden State Vector  $s_t-1$, 그리고 현재 타임 스텝의 입력으로 들어온  $y_t-1$ 이 사용되게 된다. 이 때  Context Vector $c_t$는 $s_t-1$과 인코더 Hidden State Matrix 를 이용한 Attention 메커니즘의 결과가 된다. 

![Bahdanau_Attention](https://user-images.githubusercontent.com/70866993/137129301-28753f44-751e-4658-8210-f9330ca0c9d0.png)

In [7]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W_decoder = tf.keras.layers.Dense(units)
    self.W_encoder = tf.keras.layers.Dense(units)
    self.W_combine = tf.keras.layers.Dense(1)

  def call(self, H_encoder, H_decoder):
    print("[ H_encoder ] Shape:", H_encoder.shape)

    H_encoder = self.W_encoder(H_encoder)
    print("[ W_encoder X H_encoder ] Shape:", H_encoder.shape) #인코더

    print("\n[ H_decoder ] Shape:", H_decoder.shape) #디코더
    H_decoder = tf.expand_dims(H_decoder, 1)
    H_decoder = self.W_decoder(H_decoder) #인코더의 실제 히든디코더를 한번 더 확인하도록..
    
    print("[ W_decoder X H_decoder ] Shape:", H_decoder.shape)

    score = self.W_combine(tf.nn.tanh(H_decoder + H_encoder))   #탄젠트
    print("[ Score_alignment ] Shape:", score.shape)
    
    attention_weights = tf.nn.softmax(score, axis=1)  #소프트맥스
    print("\n최종 Weight:\n", attention_weights.numpy())

    context_vector = attention_weights * H_decoder   #컨텍스벡터 어텐션가중치 *컨벡스트백터(히든디코더)
    context_vector = tf.reduce_sum(context_vector, axis=1)  # 가중치 다 더해주기

    return context_vector, attention_weights

W_size = 100

print("Hidden State를 {0}차원으로 Mapping\n".format(W_size))

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 100)

[ H_decoder ] Shape: (1, 512)
[ W_decoder X H_decoder ] Shape: (1, 1, 100)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[0.05673456]
  [0.16052398]
  [0.09991126]
  [0.10336239]
  [0.11457352]
  [0.09757707]
  [0.073319  ]
  [0.05553006]
  [0.10273167]
  [0.1357365 ]]]


### Luong Attention
- Luong Attention이 Bahdanau Attention과 가장 많이 달라진 점은 바로 Hidden State Vector를 구하는 방식이다.

[Luong_Attention] = 이름 : dot  -> $W_a$(학습 가능한 가중치행렬)  내적 추가 시 general 이라고 한다. 반면 Luong Attention은  $s_t-1$,이 아닌 $s_t$ 를 이용하여  $c_t$를 구한다. 

![Luong_Attention](https://user-images.githubusercontent.com/70866993/137130187-d3b0bc6e-bca3-49fb-be79-ce254eb4bc9f.png)


<mark> #Bahdanau 의 Score 함수와는 다르게 하나의 Weight를 사용함</mark>
![코드 차이점](https://user-images.githubusercontent.com/70866993/137131908-3d518372-bcc9-4b41-87f0-2654c9aad8c6.JPG)

In [9]:
class LuongAttention(tf.keras.layers.Layer):  # 하나의 Weight만을 사용하는 것이 특징
  def __init__(self, units):
    super(LuongAttention, self).__init__()
    self.W_combine = tf.keras.layers.Dense(units)

  def call(self, H_encoder, H_decoder):
    print("[ H_encoder ] Shape:", H_encoder.shape)

    WH = self.W_combine(H_encoder)
    print("[ W_encoder X H_encoder ] Shape:", WH.shape)

    H_decoder = tf.expand_dims(H_decoder, 1)
    alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
    print("[ Score_alignment ] Shape:", alignment.shape)

    attention_weights = tf.nn.softmax(alignment, axis=1) 
    print("\n최종 Weight:\n", attention_weights.numpy())

    attention_weights = tf.squeeze(attention_weights, axis=-1) #Bahdanau 의 Score 함수와는 다르게 하나의 Weight를 사용함
    context_vector = tf.matmul(attention_weights, H_encoder)

    return context_vector, attention_weights

emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[ H_encoder ] Shape: (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 512)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight:
 [[[1.9319450e-05]
  [3.1079210e-02]
  [7.1173120e-01]
  [1.7473552e-05]
  [6.0453255e-02]
  [1.1496807e-02]
  [6.4919405e-02]
  [3.8370891e-03]
  [1.7276463e-03]
  [1.1471859e-01]]]


### 구글이 만든 신경망 번역 시스템 GNMT (Google Nueral Machine Translation) 
- Bahdanau Attention 방식 채택, 정확도를 올리고자 Residual 노드를 넣는다 (LSTM stack에)
- <a href="https://norman3.github.io/papers/docs/google_neural_machine_translation.html" tatget="_blank"> GNMT 설명 </a>
    - 장점 : 정확도 및 학습속도 향상, Gradient Exploding / Vanishing 문제를 해결
    
- 이후에 트랜스포머 모델이 등장한다.

###  트랜스포머 모델 구조

![트랜스포머의 구조](https://user-images.githubusercontent.com/70866993/137133384-41455053-ddbc-4089-a778-42819bb902ca.jpg)


- Positional Encoding :문장에 연속성을 부여하는 새로운 방법을 제시

[참고글] 
- <a href="https://lovit.github.io/machine%20learning/2019/03/17/attention_in_nlp/" target="_blank"> sequence to sequence 에서 제안된 attention 부터, self-attention 을 이용하는 언어 모델인 BERT 까지</a> 
- <a href="https://wikidocs.net/book/2155" target="_blank"> 딥러닝을 이용한 자연어처리 입문 </a>